![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.8.Clinical_Deidentification_for_Structured_Data.ipynb)


# Clinical Deidentification for Structured Data

## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

## Colab Setup

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.5.0 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

## Getting the MYSQL jar connector

In [3]:
!wget https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/9.0.0/mysql-connector-j-9.0.0.jar

--2025-10-17 17:02:10--  https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/9.0.0/mysql-connector-j-9.0.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 104.18.19.12, 104.18.18.12, 2606:4700::6812:130c, ...
Connecting to repo1.maven.org (repo1.maven.org)|104.18.19.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2590119 (2.5M) [application/java-archive]
Saving to: ‘mysql-connector-j-9.0.0.jar’

mysql-connector-j-9 100%[===================>]   2.47M  --.-KB/s    in 0.04s   

2025-10-17 17:02:10 (58.1 MB/s) - ‘mysql-connector-j-9.0.0.jar’ saved [2590119/2590119]



## Starting the Spark session

In [4]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.pipeline_tracer import PipelineTracer
from sparknlp_jsl.pipeline_output_parser import PipelineOutputParser

from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np


# Define your Spark parameters
params = {
    "spark.driver.memory": "16G",
    "spark.kryoserializer.buffer.max": "2000M",
    "spark.driver.maxResultSize": "2000M",
    "spark.jars": "/content/mysql-connector-j-9.0.0.jar"  # Path to the downloaded JAR
}

# Start the Spark NLP session with Spark NLP JSL
spark = sparknlp_jsl.start(secret=license_keys["SECRET"],
                           params=params)

# Print version details
print("Spark NLP Version:", sparknlp.version())
print("Spark NLP_JSL Version:", sparknlp_jsl.version())

spark

Spark NLP Version: 6.1.3
Spark NLP_JSL Version: 6.1.1


In [5]:
# if you want to start the session with custom params as in start function above
from pyspark.sql import SparkSession

def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")

    return builder.getOrCreate()

#spark = start(SECRET)

# Structured Deidentification for File-Based Data

In [6]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/hipaa-table-001.txt

df = spark.read.format("csv") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("hipaa-table-001.txt")

df = df.withColumnRenamed("PATIENT","NAME")
df.show(truncate=False)

+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
|NAME           |DOB       |AGE|ADDRESS                                             |ZIPCODE|TEL           |SBP|DBP|
+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
|Cecilia Chapman|04/02/1935|83 |711-2880 Nulla St. Mankato Mississippi              |69200  |(257) 563-7401|101|42 |
|Iris Watson    |03/10/2009|9  |P.O. Box 283 8562 Fusce Rd. Frederick Nebraska      |20620  |(372) 587-2335|159|122|
|Bryar Pitts    |11/01/1921|98 |5543 Aliquet St. Fort Dodge GA                      |20783  |(717) 450-4729|149|52 |
|Theodore Lowe  |13/02/2002|16 |Ap #867-859 Sit Rd. Azusa New York                  |39531  |(793) 151-6230|134|115|
|Calista Wise   |20/08/1942|76 |7292 Dictum Av. San Antonio MI                      |47096  |(492) 709-6392|139|78 |
|Kyla Olsen     |12/05/1973|45 |Ap #651-8679 Sodales Av. Tamunin

In [7]:
from sparknlp_jsl.structured_deidentification import StructuredDeidentification

In [8]:
obfuscator = StructuredDeidentification(spark,{"NAME":"PATIENT","AGE":"AGE"}, obfuscateRefSource = "faker")
obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.show(truncate=False)

+--------------------+----------+-----+----------------------------------------------------+-------+--------------+---+---+
|NAME                |DOB       |AGE  |ADDRESS                                             |ZIPCODE|TEL           |SBP|DBP|
+--------------------+----------+-----+----------------------------------------------------+-------+--------------+---+---+
|[Kristan First]     |04/02/1935|[95] |711-2880 Nulla St. Mankato Mississippi              |69200  |(257) 563-7401|101|42 |
|[Hurley Quiver]     |03/10/2009|[5]  |P.O. Box 283 8562 Fusce Rd. Frederick Nebraska      |20620  |(372) 587-2335|159|122|
|[Rhunette Dubois]   |11/01/1921|[99] |5543 Aliquet St. Fort Dodge GA                      |20783  |(717) 450-4729|149|52 |
|[Ronni Scurry]      |13/02/2002|[13] |Ap #867-859 Sit Rd. Azusa New York                  |39531  |(793) 151-6230|134|115|
|[Josefine Carolin]  |20/08/1942|[62] |7292 Dictum Av. San Antonio MI                      |47096  |(492) 709-6392|139|78 |
|[Georga

In [9]:
obfuscator_unique_ref_test = '''Will Perry#PATIENT
John Smith#PATIENT
Marvin MARSHALL#PATIENT
Hubert GROGAN#PATIENT
ALTHEA COLBURN#PATIENT
Kalil AMIN#PATIENT
Inci FOUNTAIN#PATIENT
Jackson WILLE#PATIENT
Jack SANTOS#PATIENT
Mahmood ALBURN#PATIENT
Marnie MELINGTON#PATIENT
Aysha GHAZI#PATIENT
Maryland CODER#PATIENT
Darene GEORGIOUS#PATIENT
Shelly WELLBECK#PATIENT
Min Kun JAE#PATIENT
Thomson THOMAS#PATIENT
Christian SUDDINBURG#PATIENT
Aberdeen#CITY
Louisburg St#STREET
France#LOC
Nick Riviera#DOCTOR
5552312#PHONE
St James Hospital#HOSPITAL
Calle del Libertador#ADDRESS
111#ID
Will#DOCTOR
20#AGE
30#AGE
40#AGE
50#AGE
60#AGE
'''

with open('obfuscator_unique_ref_test.txt', 'w') as f:
  f.write(obfuscator_unique_ref_test)

In [10]:
# obfuscateRefSource = "file"

obfuscator = StructuredDeidentification(spark,{"NAME":"PATIENT","AGE":"AGE"},
                                        obfuscateRefFile = "/content/obfuscator_unique_ref_test.txt",
                                        obfuscateRefSource = "file",
                                        columnsSeed={"NAME": 23, "AGE": 23})
obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.select("NAME","AGE").show(truncate=False)

+----------------------+----+
|NAME                  |AGE |
+----------------------+----+
|[Christian Suddinburg]|[60]|
|[Christian Suddinburg]|[30]|
|[Thomson Thomas]      |[30]|
|[Aysha Ghazi]         |[40]|
|[Jack Santos]         |[40]|
|[Mahmood Alburn]      |[40]|
|[Jackson Wille]       |[60]|
|[Maryland Coder]      |[60]|
|[Kalil Amin]          |[60]|
|[Kalil Amin]          |[20]|
|[Thomson Thomas]      |[60]|
|[Kalil Amin]          |[40]|
|[Mahmood Alburn]      |[30]|
|[Darene Georgious]    |[30]|
|[Jack Santos]         |[30]|
|[Maryland Coder]      |[60]|
|[Darene Georgious]    |[50]|
|[Maryland Coder]      |[30]|
|[Mahmood Alburn]      |[20]|
|[Thomson Thomas]      |[20]|
+----------------------+----+
only showing top 20 rows



We can **shift n days** in the structured deidentification through "days" parameter when the column is a Date.

In [11]:
df = spark.createDataFrame([
            ["Juan García", "13/02/1977", "711 Nulla St.", "140", "673 431234"],
            ["Will Smith", "23/02/1977", "1 Green Avenue.", "140", "+23 (673) 431234"],
            ["Pedro Ximénez", "11/04/1900", "Calle del Libertador, 7", "100", "912 345623"]
        ]).toDF("NAME", "DOB", "ADDRESS", "SBP", "TEL")
df.show(truncate=False)

+-------------+----------+-----------------------+---+----------------+
|NAME         |DOB       |ADDRESS                |SBP|TEL             |
+-------------+----------+-----------------------+---+----------------+
|Juan García  |13/02/1977|711 Nulla St.          |140|673 431234      |
|Will Smith   |23/02/1977|1 Green Avenue.        |140|+23 (673) 431234|
|Pedro Ximénez|11/04/1900|Calle del Libertador, 7|100|912 345623      |
+-------------+----------+-----------------------+---+----------------+



In [12]:
obfuscator = StructuredDeidentification(spark=spark,
                                        columns={"NAME": "NAME", "DOB": "DATE", "TEL": "PHONE"},
                                        columnsSeed={"NAME": 23, "DOB": 23},
                                        obfuscateRefSource="faker",
                                        days=5
                                         )

obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.show(truncate=False)

+------------------+------------+-----------------------+---+------------------+
|NAME              |DOB         |ADDRESS                |SBP|TEL               |
+------------------+------------+-----------------------+---+------------------+
|[Serene Colla]    |[18/02/1977]|711 Nulla St.          |140|[982 124321]      |
|[Deirdre Plate]   |[28/02/1977]|1 Green Avenue.        |140|[+32 (982) 124321]|
|[Ernestina Bevels]|[16/04/1900]|Calle del Libertador, 7|100|[643 210932]      |
+------------------+------------+-----------------------+---+------------------+



- `outputAsArray`: allowing control over output formatting,
- `overwrite`:  overwriting behavior
- `suffix` if overwring is false create new column
with suffixing

In [13]:
obfuscator_df = obfuscator.obfuscateColumns(df, outputAsArray=False, overwrite=False, suffix="_obfuscated")
obfuscator_df.show(truncate=False)

+-------------+----------+-----------------------+---+----------------+--------------+----------------+----------------+
|NAME         |DOB       |ADDRESS                |SBP|TEL             |DOB_obfuscated|TEL_obfuscated  |NAME_obfuscated |
+-------------+----------+-----------------------+---+----------------+--------------+----------------+----------------+
|Juan García  |13/02/1977|711 Nulla St.          |140|673 431234      |18/02/1977    |784 542345      |Serene Colla    |
|Will Smith   |23/02/1977|1 Green Avenue.        |140|+23 (673) 431234|28/02/1977    |+34 (784) 542345|Deirdre Plate   |
|Pedro Ximénez|11/04/1900|Calle del Libertador, 7|100|912 345623      |16/04/1900    |023 456734      |Ernestina Bevels|
+-------------+----------+-----------------------+---+----------------+--------------+----------------+----------------+



Key additions include options for regional date formats `region`, selective date obfuscation `keepYear`, `keepMonth`, text length preservation `keepTextSizeForObfuscation`, `fakerLengthOffset`, gender-aware name obfuscation `genderAwareness`, and HIPAA-compliant age anonymization `ageRangesByHipaa`.  

In [14]:
obfuscator = StructuredDeidentification(spark,
                                        columns={"NAME":"PATIENT","DOB":"DATE","TEL":"PHONE"},
                                        columnsSeed={"PATIENT": 23, "DOB": 23, "TEL": 23},
                                        obfuscateRefSource = "faker",
                                        days=360,
                                        region="eu",
                                        keepYear=True,
                                        keepTextSizeForObfuscation=True
                                        )
obfuscator_df = obfuscator.obfuscateColumns(df, outputAsArray=False, overwrite=False, suffix="_obfuscated")
obfuscator_df.show(truncate=False)

+-------------+----------+-----------------------+---+----------------+--------------+----------------+---------------+
|NAME         |DOB       |ADDRESS                |SBP|TEL             |DOB_obfuscated|TEL_obfuscated  |NAME_obfuscated|
+-------------+----------+-----------------------+---+----------------+--------------+----------------+---------------+
|Juan García  |13/02/1977|711 Nulla St.          |140|673 431234      |08/02/1977    |548 780987      |Inez Brandy    |
|Will Smith   |23/02/1977|1 Green Avenue.        |140|+23 (673) 431234|18/02/1977    |+98 (548) 780987|Emry Leaks     |
|Pedro Ximénez|11/04/1900|Calle del Libertador, 7|100|912 345623      |06/04/1900    |209 876598      |Alene Moorman  |
+-------------+----------+-----------------------+---+----------------+--------------+----------------+---------------+



# Structured Deidentification for Relational Database

Let's install the MYSQL server dependencies

In [ ]:
!apt-get update -y -q
!apt-get install -y -q mysql-server
!pip install -q pyspark pymysql

Now let's start the mysql service

In [16]:
!service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


Define the SQL commands to reset the root password


In [17]:
reset_password_sql = """
ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root';
FLUSH PRIVILEGES;
"""

# Write the SQL commands to a file
with open("reset_password.sql", "w") as file:
    file.write(reset_password_sql)

Let's restart the MySQL service to take effect

In [18]:
!service mysql stop
# Start MySQL with the init file in the background
import time

!nohup mysqld --init-file=/content/reset_password.sql > mysql_init.log 2>&1 &
time.sleep(10)

 * Stopping MySQL database server mysqld
   ...done.


In [19]:
# Remove the init file
!rm /content/reset_password.sql
# Create the /nonexistent directory
!mkdir /nonexistent
!service mysql start

 * Starting MySQL database server mysqld
   ...done.


Now, we will create a clinical database to be deidentified, called `healthcare_db` with two tables: patients and appointments with a relational join between them:


In [20]:
# Create the healthcare database
!mysql -u root -proot -e "CREATE DATABASE healthcare_db;"

# Create the patients table
!mysql -u root -proot -e "USE healthcare_db; CREATE TABLE patients ( \
    patient_id INT AUTO_INCREMENT PRIMARY KEY, \
    name VARCHAR(255) NOT NULL, \
    address TEXT, \
    ssn CHAR(11) UNIQUE NOT NULL, \
    email VARCHAR(255) UNIQUE NOT NULL, \
    dob DATE NOT NULL, \
    age INT NOT NULL \
);"

# Create the appointments table with a foreign key reference to patients
!mysql -u root -proot -e "USE healthcare_db; CREATE TABLE appointments ( \
    appointment_id INT AUTO_INCREMENT PRIMARY KEY, \
    patient_id INT NOT NULL, \
    doctor_name VARCHAR(255) NOT NULL, \
    appointment_date DATE NOT NULL, \
    reason TEXT, \
    FOREIGN KEY (patient_id) REFERENCES patients(patient_id) \
    ON DELETE CASCADE ON UPDATE CASCADE \
);"

# Insert fake data into the patients table
!mysql -u root -proot -e "USE healthcare_db; INSERT INTO patients (name, address, ssn, email, dob, age) VALUES \
    ('John Doe', '123 Main St, Springfield', '123-45-6789', 'john.doe@example.com', '1985-04-15', 38), \
    ('Jane Smith', '456 Elm St, Shelbyville', '987-65-4321', 'jane.smith@example.com', '1990-07-20', 33);"

# Insert fake data into the appointments table
!mysql -u root -proot -e "USE healthcare_db; INSERT INTO appointments (patient_id, doctor_name, appointment_date, reason) VALUES \
    (1, 'Dr. Emily Carter', '2024-01-15', 'Annual Checkup'), \
    (2, 'Dr. Sarah Johnson', '2024-02-10', 'Flu Symptoms'), \
    (1, 'Dr. Emily Carter', '2024-02-15', 'Follow-up Visit'), \
    (1, 'Dr. James Wilson', '2024-03-20', 'Routine Blood Test');"

mysql: [Warning] Using a password on the command line interface can be insecure.
mysql: [Warning] Using a password on the command line interface can be insecure.
mysql: [Warning] Using a password on the command line interface can be insecure.
mysql: [Warning] Using a password on the command line interface can be insecure.
mysql: [Warning] Using a password on the command line interface can be insecure.


Reading the relational database as spark df for further processing:

In [21]:
jdbc_url = "jdbc:mysql://localhost:3306/healthcare_db"
df = spark.read.format("jdbc").options(
    url=jdbc_url,
    driver="com.mysql.cj.jdbc.Driver",
    dbtable="appointments",
    user="root",
    password="root"
).load()

df.show()

+--------------+----------+-----------------+----------------+------------------+
|appointment_id|patient_id|      doctor_name|appointment_date|            reason|
+--------------+----------+-----------------+----------------+------------------+
|             1|         1| Dr. Emily Carter|      2024-01-15|    Annual Checkup|
|             2|         2|Dr. Sarah Johnson|      2024-02-10|      Flu Symptoms|
|             3|         1| Dr. Emily Carter|      2024-02-15|   Follow-up Visit|
|             4|         1| Dr. James Wilson|      2024-03-20|Routine Blood Test|
+--------------+----------+-----------------+----------------+------------------+



Now, let's import the database_deidentification utility module then setup the relational database deidentification options

In [22]:
from sparknlp_jsl.utils.database_deidentification import *

In [23]:
config = {
    "db_config": {
        "host": "localhost",
        "user": "root",
        "password": "root",
        "database": "healthcare_db"
    },
    "deid_options": {
        "days_to_shift": 10,
        "age_groups": {
            "child": (0, 12),
            "teen": (13, 19),
            "adult": (20, 64),
            "senior": (65, 90)
        },
        "pk_fk_shift_value": 100,
        "use_hipaa": False,
        "output_path": "deidentified_output/"
    },
    "logging": {
        "level": "INFO",
        "file": "deidentification.log"
    }
}

Performing the structured deidentification for the clinical relational database and showing the results

In [24]:
deidentifier = RelationalDBDeidentification(spark, config)
deidentifier.deidentify()

Original table: appointments
+--------------+----------+-----------------+----------------+------------------+
|appointment_id|patient_id|doctor_name      |appointment_date|reason            |
+--------------+----------+-----------------+----------------+------------------+
|1             |1         |Dr. Emily Carter |2024-01-15      |Annual Checkup    |
|2             |2         |Dr. Sarah Johnson|2024-02-10      |Flu Symptoms      |
|3             |1         |Dr. Emily Carter |2024-02-15      |Follow-up Visit   |
|4             |1         |Dr. James Wilson |2024-03-20      |Routine Blood Test|
+--------------+----------+-----------------+----------------+------------------+

De-identified table: appointments
+--------------+----------+-----------+----------------+------------------+
|appointment_id|patient_id|doctor_name|appointment_date|reason            |
+--------------+----------+-----------+----------------+------------------+
|101           |101       |*****      |2024-01-25   